# Testes da rotina que gera os vetores de entrada com base em uma série temporal

## 1. Importando as bibliotecas necessárias

### Bibliotecas obrigatórias

In [1]:
import numpy as np

### Bibliotecas não-obrigatórias

In [2]:
import sys 
sys.path.insert(0, '../../../scripts')
import timeseries as ts

## 2. Definindo a classe (debug)

Não é necessário executar essa linha se você estiver importando o arquivo com a classe.

In [3]:
class SerieTemporal:

    def __init__(self, dados, K, L):
        """
        Descrição:
        ----------
        Construtor da classe 'SerieTemporal'

        Parâmetros:
        -----------
        dados: np.ndarray
            Conjunto de valores da série temporal
        K: int
            O número de entradas utilizado para a predição
        L: int
            O passo de predição     
            
        Retorna:
        --------
        Nada
        """

        if not type(dados) is np.ndarray:
            raise TypeError("Os dados devem ser um array do numpy!")        

        if (K <= 0):
            raise ValueError("O hiperparâmetro 'K' deve ser um inteiro positivo!")

        if (L > len(dados)):
            raise ValueError("L deve ser menor que o número de dados temporais!")

        self.__dados = dados
        self.__K = K
        self.__L = L
        self._matriz_entrada = np.array([])
        self._matriz_saida = np.array([])
        pass

    def _criar_vetores(self, indice):
        """
        Descrição:
        ----------
        Função interna para criar os vetores de entrada e saída para as etapas de treinamento e teste para uma série temporal
    
        Parâmetros:
        -----------
        indice: int
            A posicao a partir da qual se deseja prever algum valor
            
        Retorna:
        --------
        Os vetores de entrada e saída para o índice escolhido no formato np.ndarray
        """

        K, L = self.__K, self.__L
        dados = self.__dados
    
        # checa se a partir da posição atual podemos criar um vetor de amostras dado um K
        if ((indice + 1) < (K - 1)):
            raise ValueError("(indice + 1) = "+str(indice + 1)+" deve ser maior ou igual a (K - 1) = "+str(K - 1)+" !")
        
        # checa se o valor que queremos prever (que vai ser armazenado na matriz de saida), está dentro da série temporal
        if ((indice+L) > (len(dados)-1)):
            raise ValueError("O passo de predição (L = "+str(L)+") somado com o índice atual (indice = "+str(indice)+") não deve estourar o número de dados na série temporal!")
    
        vetor_entrada = np.array(dados[(indice-(K-1)):(indice+1)])
        vetor_saida = np.array(dados[indice+L])

        return vetor_entrada, vetor_saida    

    def dividir_treino_teste(self, tam_teste):
        """
        Descrição:
        ----------
        Função para selecionar os tam_teste*len(matriz_entrada) últimos dados das matrizes para o teste

        Parâmetros:
        -----------
        tam_teste: float
            Proporção de dados que iremos separar para o teste. Deve ser entre 0.0 e 1.0
            
        Retorna:
        --------
        O conjunto de teste e treinamento para a proporção solicitada no formato np.ndarray
        """
        
        if ((tam_teste < 0.0) | (tam_teste > 1.0)):
            raise ValueError("A proporção dos dados de teste deve ser entre 0.0 e 1.0!")
            
        matriz_entrada = self._matriz_entrada
        matriz_saida = self._matriz_saida
            
        tam_treino = 1.0 - tam_teste
        n_dados = len(matriz_saida)

        X_treino = np.array(matriz_entrada[:int(tam_treino*n_dados)])
        X_teste = np.array(matriz_entrada[int(n_dados*(1 - tam_teste)):])
        y_treino = np.array(matriz_saida[:int(tam_treino*n_dados)])
        y_teste = np.array(matriz_saida[int(n_dados*(1 - tam_teste)):])
        
        return (X_treino, X_teste,
                y_teste, y_teste)

    def criar_matrizes(self):
        """
        Descrição:
        ----------
        Função para criar as matrizes com os vetores de entrada e saída para as etapas de treinamento e teste
    
        Parâmetros:
        -----------
        Nenhum
        
        Retorna:
        --------
        Nada
        """
    
        K, L = self.__K, self.__L
        num_dados = len(self.__dados)
    
        for indice in range((K-1), (num_dados-L)):
            vetor_entrada, vetor_saida = self._criar_vetores(indice)
        
            if (len(self._matriz_entrada) == 0):
                self._matriz_entrada = vetor_entrada
            else:
                self._matriz_entrada = np.vstack((self._matriz_entrada, vetor_entrada))
                
            if (len(self._matriz_saida) == 0):
                self._matriz_saida = np.array([vetor_saida])
            else:
                self._matriz_saida = np.vstack((self._matriz_saida, vetor_saida))
        pass

## 3. Testando

In [4]:
serie_temporal = SerieTemporal(dados=np.array(range(0, 100)), K=4, L=1)

In [5]:
serie_temporal.criar_matrizes()

In [6]:
X_treino, X_teste, y_treino, y_teste = serie_temporal.dividir_treino_teste(tam_teste=0.35)

In [7]:
print("Matriz de entrada do conjunto de treinamento:\n", X_treino, "\n")
print("Matriz de saída do conjunto de treinamento:\n", y_treino, "\n")
print("Matriz de entrada do conjunto de teste:\n", X_teste, "\n")
print("Matriz de saída do conjunto de teste:\n", y_teste)

Matriz de entrada do conjunto de treinamento:
 [[ 0  1  2  3]
 [ 1  2  3  4]
 [ 2  3  4  5]
 [ 3  4  5  6]
 [ 4  5  6  7]
 [ 5  6  7  8]
 [ 6  7  8  9]
 [ 7  8  9 10]
 [ 8  9 10 11]
 [ 9 10 11 12]
 [10 11 12 13]
 [11 12 13 14]
 [12 13 14 15]
 [13 14 15 16]
 [14 15 16 17]
 [15 16 17 18]
 [16 17 18 19]
 [17 18 19 20]
 [18 19 20 21]
 [19 20 21 22]
 [20 21 22 23]
 [21 22 23 24]
 [22 23 24 25]
 [23 24 25 26]
 [24 25 26 27]
 [25 26 27 28]
 [26 27 28 29]
 [27 28 29 30]
 [28 29 30 31]
 [29 30 31 32]
 [30 31 32 33]
 [31 32 33 34]
 [32 33 34 35]
 [33 34 35 36]
 [34 35 36 37]
 [35 36 37 38]
 [36 37 38 39]
 [37 38 39 40]
 [38 39 40 41]
 [39 40 41 42]
 [40 41 42 43]
 [41 42 43 44]
 [42 43 44 45]
 [43 44 45 46]
 [44 45 46 47]
 [45 46 47 48]
 [46 47 48 49]
 [47 48 49 50]
 [48 49 50 51]
 [49 50 51 52]
 [50 51 52 53]
 [51 52 53 54]
 [52 53 54 55]
 [53 54 55 56]
 [54 55 56 57]
 [55 56 57 58]
 [56 57 58 59]
 [57 58 59 60]
 [58 59 60 61]
 [59 60 61 62]
 [60 61 62 63]
 [61 62 63 64]] 

Matriz de saída do c